<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item MAE</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit.metrics.predict import mae
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # train the algorithm with the training dataset
    # Now we run the recommender:
    preds = batch.predict(fittable, test)
    # add the algorithm name for analyzability
    return preds

In [2]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,center=center,aggregate='weighted-average') # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)

    metric = mae(all_recs['prediction'], all_recs['rating'])
    return metric

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric],',')
    return {'loss': metric, 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[94.88977034751147, 'True', 0.7178833392699701]                                                                        
,                                                                                                                      
[59.85443550383254, 'True', 0.7142598319495012]                                                                        
,                                                                                                                      
[82.41723730432835, 'False', 0.7194846518862236]                                                                       
,                                                                                                                      
[87.72772901892405, 'False', 0.7141456532326849]                                                                       
,                                                                                                                      
[48.963257492741285, 'True', 0.710681386

[33.006252985958824, 'False', 0.7052296277223552]                                                                      
,                                                                                                                      
[57.15083986847468, 'False', 0.706921887898611]                                                                        
,                                                                                                                      
[4.397848896656253, 'False', 0.7413795190317738]                                                                       
,                                                                                                                      
[13.760535657656584, 'False', 0.7073205500495492]                                                                      
,                                                                                                                      
[8.34366275372868, 'True', 0.71311873154

[40.3787183056647, 'False', 0.708499658827107]                                                                         
,                                                                                                                      
[30.600875748762487, 'False', 0.7087686463677803]                                                                      
,                                                                                                                      
[33.86120424770791, 'False', 0.7122182080972791]                                                                       
,                                                                                                                      
[46.03213598704396, 'False', 0.7088565843634517]                                                                       
,                                                                                                                      
[17.964204208873014, 'False', 0.70697914

In [7]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','MAE']
metric

,neighbours,center,MAE
0,94.889770,True,0.717883
1,59.854436,True,0.714260
2,82.417237,False,0.719485
3,87.727729,False,0.714146
4,48.963257,True,0.710681
...,...,...,...
95,20.177300,False,0.708079
96,44.595499,True,0.710110
97,2.685787,False,0.796162
98,63.131112,False,0.712712


In [8]:
metric.sort_values(by=['MAE'],ascending=True).head(10)

,neighbours,center,MAE
49,22.639671,False,0.698998
99,14.200900,False,0.701960
55,34.653927,False,0.702519
52,30.843853,False,0.702925
28,27.465317,False,0.703624
84,15.857115,False,0.703999
5,22.617484,False,0.704179
63,16.013920,False,0.704389
27,27.412927,False,0.704865
23,16.795116,False,0.704901


In [9]:
metric.to_csv('metric_MAE.csv', index=False)